In [ ]:
!pip install --upgrade pip
!pip install fosforml numpy pandas matplotlib scikit-learn seaborn python-dateutil
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml 
!pip install fosforio
!pip install refractio
!pip install refractml

In [ ]:
!pip install seaborn scipy xgboost pandas dice-ml tabulate numpy scikit-learn pandas-profiling plotly matplotlib scipy statsmodels seaborn pydantic-settings

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, OneHotEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
from scipy.stats.mstats import winsorize
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
import joblib
from fosforml import *
from fosforml.constants import MLModelFlavours
import warnings
warnings.filterwarnings("ignore", message="numpy.dtype size changed")
warnings.filterwarnings("ignore", message="numpy.ufunc size changed")

In [25]:
from refractio import snowflake
snowflake.get_connection(connection_name="HR_ANALYTICS_V")

Exception occurred in getting snowflake connection: 'connectionSources'


In [26]:
df = snowflake.get_dataframe("HR_ANALYTICS_V")
df

,SALARY,SENIORITY,TENURE_MONTHS,MONTHS_AFTER_COLLEGE,BIRTH_YEAR,MAPPED_ROLE_CLEAN,SEX,ETHNICITY,HOSPITAL_TYPE,HOSPITAL_OWNERSHIP,...,CITY,STATE,DISTANCE,DEGREE_CLEAN,SCHOOL_ENDDATE,JOB_STARTDATE,JOB_ENDDATE,USER_ID,OVERTIME_HOURS,CHURN
0,53239.07,4,5,358,1965,nurse,M,White,Acute Care Hospitals,Voluntary non-profit - Private,...,Yakima,WA,>10 miles,Associates Degree,1987-01-01,2016-11-01,2017-06-01,147501806,13,1
1,29277.97,1,10,18,1993,nurse,F,Multiple,Acute Care Hospitals,Voluntary non-profit - Church,...,Tampa,FL,>10 miles,Undergraduate Diploma,2014-01-01,2015-07-01,2016-06-01,46143395,15,1
2,43763.54,2,20,6,1985,technologist,F,White,Acute Care Hospitals,Voluntary non-profit - Church,...,Columbus,OH,5-10 miles,Bachelors Degree,2010-01-31,2010-07-01,2012-05-01,567514867,0,1
3,24728.57,1,14,16,1992,nurse,M,White,Acute Care Hospitals,Proprietary,...,Miami,FL,>10 miles,Undergraduate Diploma,2013-01-31,2014-05-01,2015-09-01,65551252,9,0
4,46819.01,2,43,17,1983,technologist,M,White,Acute Care Hospitals,Voluntary non-profit - Private,...,Cleveland,OH,2-5 miles,Bachelors Degree,2008-01-01,2009-06-01,None,84119911,13,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,30934.15,1,2,8,1996,nurse,M,Multiple,Psychiatric,Voluntary non-profit - Private,...,Orlando,FL,>10 miles,Other,2019-01-31,2019-09-01,2020-02-01,163926894,2,1
299996,69288.47,2,7,63,1996,nurse,F,API,Acute Care Hospitals,Proprietary,...,Tyler,TX,>10 miles,Associates Degree,2014-01-31,2019-04-01,2020-02-01,313934824,3,1
299997,53051.71,1,10,11,1991,emt,F,Multiple,Acute Care Hospitals,Voluntary non-profit - Private,...,Phoenix,AZ,5-10 miles,Bachelors Degree,2018-01-01,2018-12-01,None,812455262,8,0
299998,72186.39,2,13,8,1995,technologist,F,Black,Acute Care Hospitals,Voluntary non-profit - Other,...,Chicago,IL,<2 miles,Bachelors Degree,2020-01-01,2020-09-01,None,713192325,9,0


In [27]:
print(df.isnull().sum())

SALARY                      0
SENIORITY                   0
TENURE_MONTHS               0
MONTHS_AFTER_COLLEGE        0
BIRTH_YEAR                  0
MAPPED_ROLE_CLEAN           0
SEX                         0
ETHNICITY                   0
HOSPITAL_TYPE               0
HOSPITAL_OWNERSHIP          0
COMPANY_NAME                0
CITY                        0
STATE                       0
DISTANCE                    0
DEGREE_CLEAN                0
SCHOOL_ENDDATE              0
JOB_STARTDATE               0
JOB_ENDDATE             90076
USER_ID                     0
OVERTIME_HOURS              0
CHURN                       0
dtype: int64


In [28]:
df = df.dropna()

In [29]:
print(df.isnull().sum())

SALARY                  0
SENIORITY               0
TENURE_MONTHS           0
MONTHS_AFTER_COLLEGE    0
BIRTH_YEAR              0
MAPPED_ROLE_CLEAN       0
SEX                     0
ETHNICITY               0
HOSPITAL_TYPE           0
HOSPITAL_OWNERSHIP      0
COMPANY_NAME            0
CITY                    0
STATE                   0
DISTANCE                0
DEGREE_CLEAN            0
SCHOOL_ENDDATE          0
JOB_STARTDATE           0
JOB_ENDDATE             0
USER_ID                 0
OVERTIME_HOURS          0
CHURN                   0
dtype: int64


In [31]:
df = df.drop(["USER_ID", "JOB_STARTDATE", "JOB_ENDDATE", "SCHOOL_ENDDATE"], axis = 1)

KeyError: "['USER_ID', 'JOB_STARTDATE', 'JOB_ENDDATE', 'SCHOOL_ENDDATE'] not found in axis"

In [8]:
CATEGORICAL_COLUMNS = ["MAPPED_ROLE_CLEAN","SEX", "ETHNICITY","HOSPITAL_TYPE", "HOSPITAL_OWNERSHIP","COMPANY_NAME","CITY","STATE","DISTANCE", 
                       "DEGREE_CLEAN"]
NUMERICAL_COLUMNS = ["SALARY", "SENIORITY", "TENURE_MONTHS", "MONTHS_AFTER_COLLEGE", "BIRTH_YEAR", "OVER_TIME_HOURS",]
LABEL_COLUMNS = ["CHURN"]
OUTPUT_COLUMNS = ["PREDICTION"]

In [9]:
# Filter feature columns
feature_columns = CATEGORICAL_COLUMNS + NUMERICAL_COLUMNS

In [10]:
X = df[feature_columns]
y = df[LABEL_COLUMNS].values.ravel()

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OrdinalEncoder, MinMaxScaler
from xgboost import XGBClassifier
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
 
# Define transformers
categorical_transformer = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
)
 
numerical_transformer = make_pipeline(
    SimpleImputer(strategy='mean'),
    MinMaxScaler(clip=True)
)
 
# Create preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', categorical_transformer, CATEGORICAL_COLUMNS),
        ('num', numerical_transformer, NUMERICAL_COLUMNS)
    ]
)
 

In [17]:
# Step 6: Train and Compare Multiple Models
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier
 
models = {
    "RandomForest": RandomForestClassifier(),
    "LogisticRegression": LogisticRegression(max_iter=1000),
    "XGBoost": XGBClassifier(eval_metric="logloss", use_label_encoder=False)
}
results = {}

In [19]:
for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
y_prob = pipeline.predict_proba(X_test)[:, 1]

In [21]:
from sklearn.metrics import classification_report, log_loss, roc_auc_score
accuracy = accuracy_score(y_test, y_pred)
logloss = log_loss(y_test, y_prob)
roc_auc = roc_auc_score(y_test, y_prob)

In [23]:
  results[model_name] = {
        "accuracy": accuracy,
        "logloss": logloss,
        "roc_auc": roc_auc,
        "model": pipeline
    }
    
print(f"Model: {model_name}")
print(f"Accuracy: {accuracy}")
print(f"Log Loss: {logloss}")
print(f"ROC AUC Score: {roc_auc}")
print("="*30)

Model: XGBoost
Accuracy: 0.5002382086707956
Log Loss: 0.7003753569904368
ROC AUC Score: 0.5026587989402631


In [24]:
# Step 7: Select the Best Model
best_model_name = max(results, key=lambda k: results[k]['roc_auc'])
best_model = results[best_model_name]['model']
 
print(f"Best Model: {best_model_name} with ROC AUC Score: {results[best_model_name]['roc_auc']}")

Best Model: XGBoost with ROC AUC Score: 0.5026587989402631
